In [2]:
import os
os.chdir('../')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
f
data_train_path = "Dataset/data_train.csv"

URM_all_dataframe = pd.read_csv(data_train_path)
print(URM_all_dataframe.head(10))

   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
4        0      211   1.0
5        0      232   1.0
6        0      282   1.0
7        0      453   1.0
8        0      458   1.0
9        0      491   1.0


In [14]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 425 ( 1.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 182 ( 0.5%) Users that have less than 1 test interactions


In [16]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

def objective_function_2(trial):

    params = {
        'topK': trial.suggest_int('topK', 5, 1000),
        # 'l1_ratio': trial.suggest_float('l1_ratio', 1e-5, 1, log=True),
        # 'alpha': trial.suggest_float('alpha', 1e-3, 1),
    }


    MAP = 0
    recommender = SLIMElasticNetRecommender(URM_train)
    recommender.fit(**params)

    result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

    MAP = result_dict.loc[10]["MAP"]
    
    return MAP

In [17]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/Slimnet_parallel.csv", index = False)

In [ ]:
import optuna

optuna_study = optuna.create_study(study_name="parallelSlimNet2", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function_2,
                      callbacks=[save_results],
                      n_trials = 100,
                      n_jobs=6)